In [2]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 2.9 MB/s eta 0:00:00


In [4]:
import csv
from transformers import MusicgenForConditionalGeneration, AutoProcessor
import torch
from IPython.display import Audio
import emoji

# Function to generate and load emoji descriptions into a dictionary using the emoji library
def generate_emoji_descriptions():
    emoji_list = emoji.EMOJI_DATA  # This is the correct way to access emoji data in some versions.
    emoji_to_description = {char: desc['en'].replace(':', '').replace('_', ' ').capitalize() for char, desc in emoji_list.items()}

    with open('emoji_music_descriptions.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Emoji(s)', 'Description'])  # Write header
        for emoji_char, desc in emoji_to_description.items():
            writer.writerow([emoji_char, desc])

    return emoji_to_description

# Load the emoji to description mappings from the CSV
csv_file_path = 'emoji_music_descriptions.csv'
emoji_to_description = generate_emoji_descriptions()  # Generate and load descriptions

# Define the model and processor
model_name = "facebook/musicgen-small"
model = MusicgenForConditionalGeneration.from_pretrained(model_name)
processor = AutoProcessor.from_pretrained(model_name)

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate music based on input
def generate_music(input_text, emoji_to_description):
    description = emoji_to_description.get(input_text, input_text)
    inputs = processor(text=[description], return_tensors="pt", padding=True).to(device)
    audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=1000).cpu().numpy()[0]
    return audio_values

# Function to play audio
def play_audio(audio_values):
    sampling_rate = model.config.audio_encoder.sampling_rate
    return Audio(audio_values, rate=sampling_rate)

# Input and music generation
prompt = input("Enter your emoji or text: ")  # Get input from the user
if prompt.lower() != 'exit':
    audio_values = generate_music(prompt, emoji_to_description)
    audio_player = play_audio(audio_values)
    display(audio_player)
else:
    print("Exiting...")


Enter your emoji or text: 😃 in paino
